[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/geemap/blob/master/docs/workshops/Alaska_2024_Part2.ipynb)

**Geospatial Cloud Computing with the GEE Python API - Part 2**

-   Notebook: <https://geemap.org/workshops/Alaska_2024_Part2>
-   Earth Engine: <https://earthengine.google.com>
-   Geemap: <https://geemap.org>

## Introduction

This notebook contains the materials for the first part of the workshop **Geospatial Cloud Computing with the GEE Python API** at the University of Alaska Fairbanks.

This workshop provides an introduction to cloud-based geospatial analysis using the Earth Engine Python API. Attendees will learn the basics of Earth Engine data types and how to visualize, analyze, and export Earth Engine data in a Jupyter environment with geemap. In addition, attendees will learn how to develop and deploy interactive Earth Engine web apps with Python. Through practical examples and hands-on exercises, attendees will enhance their learning experience. During each hands-on session, attendees will walk through Jupyter Notebook examples on Google Colab with the instructors. At the end of each session, they will complete a hands-on exercise to apply the knowledge they have learned.

### Agenda

The workshop is divided into three parts. The second part will cover the following topics:

-   Processing of vector data (shapefiles, json, conversion from one format to another)
-   Processing of raster data: extract pixel value, raster calculator, zonal statistics etc.
-   Working with local geospatial data in Geemap
-   Accessing Cloud Optimized GeoTIFF (e.g. from NASA DAACs like EARTHDATA), Esri feature and map services
-   Exporting EE Image and Feature data
-   Creating timelapse animations using Landsat or Sentinel 2 for Alaska
-   Time series analysis: Forest cover change for a test site in Alaska (e.g. Bonanza Creek LTER or Caribou-Poker Creeks Research Watershed)

### Prerequisites

-   To use geemap and the Earth Engine Python API, you must [register](https://code.earthengine.google.com/register) for an Earth Engine account and follow the instructions [here](https://docs.google.com/document/d/1ZGSmrNm6_baqd8CHt33kIBWOlvkh-HLr46bODgJN1h0/edit?usp=sharing) to create a Cloud Project. Earth Engine is free for [noncommercial and research use](https://earthengine.google.com/noncommercial). To test whether you can use authenticate the Earth Engine Python API, please run [this notebook](https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/geemap_colab.ipynb) on Google Colab.

### Import libraries

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

## Processing of vector data

### From GeoJSON

In [ ]:
in_geojson = "https://github.com/gee-community/geemap/blob/master/examples/data/countries.geojson"
m = geemap.Map()
fc = geemap.geojson_to_ee(in_geojson)
m.add_layer(fc.style(**{"color": "ff0000", "fillColor": "00000000"}), {}, "Countries")
m

### From Shapefile

In [ ]:
url = "https://github.com/gee-community/geemap/blob/master/examples/data/countries.zip"
geemap.download_file(url, overwrite=True)

In [ ]:
in_shp = "countries.shp"
fc = geemap.shp_to_ee(in_shp)

In [ ]:
m = geemap.Map()
m.add_layer(fc, {}, "Countries")
m

### From GeoDataFrame

In [ ]:
import geopandas as gpd

gdf = gpd.read_file(in_shp)
fc = geemap.gdf_to_ee(gdf)

In [ ]:
m = geemap.Map()
m.add_layer(fc, {}, "Countries")
m

### To GeoJSON

In [ ]:
m = geemap.Map()
states = ee.FeatureCollection("TIGER/2018/States")
fc = states.filter(ee.Filter.eq("NAME", "Alaska"))
m.add_layer(fc, {}, "Alaska")
m.center_object(fc, 4)
m

In [ ]:
geemap.ee_to_geojson(fc, filename="Alaska.geojson")

### To Shapefile

In [ ]:
geemap.ee_to_shp(fc, filename="Alaska.shp")

### To GeoDataFrame

In [ ]:
gdf = geemap.ee_to_gdf(fc)
gdf

In [ ]:
gdf.explore()

### To DataFrame

In [ ]:
df = geemap.ee_to_df(fc)
df

### To CSV

In [ ]:
geemap.ee_to_csv(fc, filename="Alaska.csv")

## Processing of raster data

### Extract pixel values

#### Extracting values to points

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

dem = ee.Image("USGS/SRTMGL1_003")
landsat7 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")

vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}

Map.addLayer(
    landsat7,
    {"bands": ["B4", "B3", "B2"], "min": 20, "max": 200, "gamma": 2},
    "Landsat 7",
)
Map.addLayer(dem, vis_params, "SRTM DEM", True, 1)
Map

In [ ]:
in_shp = "us_cities.shp"
url = "https://github.com/giswqs/data/raw/main/us/us_cities.zip"
geemap.download_file(url)

In [ ]:
in_fc = geemap.shp_to_ee(in_shp)
Map.addLayer(in_fc, {}, "Cities")

In [ ]:
geemap.extract_values_to_points(in_fc, dem, out_fc="dem.shp")

In [ ]:
geemap.shp_to_gdf("dem.shp")

In [ ]:
geemap.extract_values_to_points(in_fc, landsat7, "landsat.csv")

In [ ]:
geemap.csv_to_df("landsat.csv")

#### Extracting pixel values along a transect

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("TERRAIN")

image = ee.Image("USGS/SRTMGL1_003")
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}
Map.addLayer(image, vis_params, "SRTM DEM", True, 0.5)
Map

In [ ]:
line = Map.user_roi
if line is None:
    line = ee.Geometry.LineString(
        [[-120.2232, 36.3148], [-118.9269, 36.7121], [-117.2022, 36.7562]]
    )
    Map.addLayer(line, {}, "ROI")
Map.centerObject(line)

In [ ]:
reducer = "mean"
transect = geemap.extract_transect(
    image, line, n_segments=100, reducer=reducer, to_pandas=True
)
transect

In [ ]:
geemap.line_chart(
    data=transect,
    x="distance",
    y="mean",
    markers=True,
    x_label="Distance (m)",
    y_label="Elevation (m)",
    height=400,
)

In [ ]:
transect.to_csv("transect.csv")

### Zonal statistics

#### Zonal statistics with an image and a feature collection

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NASA SRTM
dem = ee.Image("USGS/SRTMGL1_003")
dem_vis = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}
Map.addLayer(dem, dem_vis, "SRTM DEM")

# Add 5-year Landsat TOA composite
landsat = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")
landsat_vis = {"bands": ["B4", "B3", "B2"], "gamma": 1.4}
Map.addLayer(landsat, landsat_vis, "Landsat", False)

# Add US Census States
states = ee.FeatureCollection("TIGER/2018/States")
style = {"fillColor": "00000000"}
Map.addLayer(states.style(**style), {}, "US States")
Map

In [ ]:
out_dem_stats = "dem_stats.csv"
geemap.zonal_stats(
    dem, states, out_dem_stats, statistics_type="MEAN", scale=1000, return_fc=False
)

In [ ]:
out_landsat_stats = "landsat_stats.csv"
geemap.zonal_stats(
    landsat,
    states,
    out_landsat_stats,
    statistics_type="MEAN",
    scale=1000,
    return_fc=False,
)

#### Zonal statistics by group

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NLCD data
dataset = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019")
landcover = dataset.select("landcover")
Map.addLayer(landcover, {}, "NLCD 2019")

# Add US census states
states = ee.FeatureCollection("TIGER/2018/States")
style = {"fillColor": "00000000"}
Map.addLayer(states.style(**style), {}, "US States")

# Add NLCD legend
Map.add_legend(title="NLCD Land Cover", builtin_legend="NLCD")
Map

In [ ]:
nlcd_stats = "nlcd_stats.csv"

geemap.zonal_stats_by_group(
    landcover,
    states,
    nlcd_stats,
    statistics_type="SUM",
    denominator=1e6,
    decimal_places=2,
)

In [ ]:
nlcd_stats = "nlcd_stats_pct.csv"

geemap.zonal_stats_by_group(
    landcover,
    states,
    nlcd_stats,
    statistics_type="PERCENTAGE",
    denominator=1e6,
    decimal_places=2,
)

#### Zonal statistics with two images

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
dem = ee.Image("USGS/3DEP/10m")
vis = {"min": 0, "max": 4000, "palette": "terrain"}
Map.addLayer(dem, vis, "DEM")
Map

In [ ]:
landcover = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select("landcover")
Map.addLayer(landcover, {}, "NLCD 2019")
Map.add_legend(title="NLCD Land Cover Classification", builtin_legend="NLCD")

In [ ]:
stats = geemap.image_stats_by_zone(dem, landcover, reducer="MEAN")
stats

In [ ]:
stats.to_csv("mean.csv", index=False)

In [ ]:
geemap.image_stats_by_zone(dem, landcover, out_csv="std.csv", reducer="STD")

### Map algebra

In [ ]:
m = geemap.Map()

# Load a 5-year Landsat 7 composite 1999-2003.
landsat_1999 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")

# Compute NDVI.
ndvi_1999 = (
    landsat_1999.select("B4")
    .subtract(landsat_1999.select("B3"))
    .divide(landsat_1999.select("B4").add(landsat_1999.select("B3")))
)

vis = {"min": 0, "max": 1, "palette": "ndvi"}
m.add_layer(ndvi_1999, vis, "NDVI")
m.add_colorbar(vis, label="NDVI")
m

In [ ]:
# Load a Landsat 8 image.
image = ee.Image("LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318")

# Compute the EVI using an expression.
evi = image.expression(
    "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))",
    {
        "NIR": image.select("B5"),
        "RED": image.select("B4"),
        "BLUE": image.select("B2"),
    },
)

# Define a map centered on San Francisco Bay.
m = geemap.Map(center=[37.4675, -122.1363], zoom=9)

vis = {"min": 0, "max": 1, "palette": "ndvi"}
m.add_layer(evi, vis, "EVI")
m.add_colorbar(vis, label="EVI")
m

## Working with local geospatial data

## Accessing Cloud Optimized GeoTIFFs

## Exporting Earth Engine data

## Creating timelapse animations

## Time series analysis